# Kmeans over a set of GeoTiffs

This notebook loads a set of GeoTiffs into a **RDD** of Tiles, with each Tile being a band in the GeoTiff. Each GeoTiff file contains **SpringIndex-** or **LastFreeze-** value for one year over the entire USA.

Kmeans takes years as dimensions. Hence, the matrix has cells as rows and the years as columns. To cluster on all years, the matrix needs to be transposed. The notebook has two flavors of matrix transpose, locally by the Spark-driver or distributed using the Spark-workers. Once transposed the matrix is converted to a **RDD** of dense vectors to be used by **Kmeans** algorithm from **Spark-MLlib**. The end result is a grid where each cell has a cluster ID which is then saved into a SingleBand GeoTiff. By saving the result into a GeoTiff, the reader can plot it using a Python notebook as the one defined in the [python examples](../examples/python).

<span style="color:red">In this notebook the reader only needs to modify the variables in **Mode of Operation Setup**</span>.

## Dependencies

In [39]:
import sys.process._

import java.io.{ByteArrayInputStream, ByteArrayOutputStream, ObjectInputStream, ObjectOutputStream}


import geotrellis.proj4.CRS
import geotrellis.raster.{CellType, ArrayTile, DoubleArrayTile, Tile, UByteCellType}
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{GeoTiff, SinglebandGeoTiff}
import geotrellis.spark.io.hadoop._
import org.apache.hadoop.io._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.distributed.{CoordinateMatrix, MatrixEntry, RowMatrix}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

import org.apache.hadoop.io.{IOUtils, SequenceFile}
import org.apache.hadoop.io.SequenceFile.Writer

//Spire is a numeric library for Scala which is intended to be generic, fast, and precise.
import spire.syntax.cfor._

lastException: Throwable = null


## Mode of operation

Here the user can define the mode of operation.
* **rdd_offline_mode**: If false it means the notebook will create all data from scratch and store grid0, grid0_index, protected_extent and num_cols_rows (from grid0) into HDFS. Otherwise, these data structures are read from HDFS.
* **matrix_offline_mode**: If false it means the notebook will create a mtrix,  transposed it and save it to HDFS. Otherwise, these data structures are read from HDFS.
* **kmeans_offline_mode**: If false it means the notebook will train kmeans and run kemans and store kmeans model into HDFS. Otherwise, these data structures are read from HDFS.

It is also possible to define which directory of GeoTiffs is to be used and on which **band** to run Kmeans. The options are
* **BloomFinal** or **LeafFinal** which are multi-band (**4 bands**)
* **DamageIndex** and **LastFreeze** which are single-band and if set band_num higher, it will reset to 0

For kmeans the user can define the **number of iterations** and **number of clusters** as an inclusive range. Such range is defined using **minClusters**, **maxClusters**, and **stepClusters**. These variables will set a loop starting at **minClusters** and stopping at **maxClusters** (inclusive), iterating **stepClusters** at the time. <span style="color:red">Note that when using a range **kemans offline mode** is not possible and it will be reset to **online mode**</span>.

### Mode of Operation setup

In [40]:
//Operation mode
var rdd_offline_mode = true
var matrix_offline_mode = true
var kmeans_offline_mode = true

//GeoTiffs to be read from "hdfs:///user/hadoop/spring-index/"
var dir_path = "hdfs:///user/hadoop/spring-index/"
var offline_dir_path = "hdfs:///user/emma/spring-index/"
var geoTiff_dir = "BloomFinal"
var band_num = 4

//Kmeans number of iterations and clusters
var numIterations = 5
var minClusters = 6
var maxClusters = 10
var stepClusters = 1

rdd_offline_mode = true
matrix_offline_mode = true
kmeans_offline_mode = true
dir_path = hdfs:///user/hadoop/spring-index/
offline_dir_path = hdfs:///user/emma/spring-index/
geoTiff_dir = BloomFinal
band_num = 4
numIterations = 5
minClusters = 6
maxClusters = 10
stepClusters = 1


1


<span style="color:red">DON'T MODIFY ANY PIECE OF CODE FROM HERE ON!!!</span>.


### More of operation validation

In [58]:
//Validation, do not modify these lines.
var single_band = false
if (geoTiff_dir == "BloomFinal" || geoTiff_dir == "LeafFinal") {
    single_band = false
} else if (geoTiff_dir == "LastFreeze" || geoTiff_dir == "DamageIndex") {
    single_band = true
    if (band_num > 0) {
        println("Since LastFreezze and DamageIndex are single band, we will use band 0!!!")
        band_num  = 0
    }
} else {
    println("Directory unknown, please set either BloomFinal, LeafFinal, LastFreeze or DamageIndex!!!")
}

if (minClusters > maxClusters) {
    maxClusters = minClusters
}
if (stepClusters < 1) {
    stepClusters = 1
}

//Paths to store data structures for Offline runs
var grid0_path = offline_dir_path + geoTiff_dir + "/grid0"
var grid0_index_path = offline_dir_path + geoTiff_dir + "/grid0_index"
var grids_noNaN_path = offline_dir_path + geoTiff_dir + "/grids_noNaN"
var metadata_path = offline_dir_path + geoTiff_dir + "/metadata"
var grids_matrix_path = offline_dir_path + geoTiff_dir + "/grids_matrix"

//Check offline modes
var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

val rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grid0_path))
val matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grids_matrix_path))
                                      
if (rdd_offline_mode != rdd_offline_exists) {
    println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + rdd_offline_exists.toString())
    rdd_offline_mode = rdd_offline_exists
} 
if (matrix_offline_mode != matrix_offline_exists) {
    println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + matrix_offline_exists.toString())
    matrix_offline_mode = matrix_offline_exists
}

var num_kmeans :Int  = ((maxClusters - minClusters + 1) / stepClusters)
var kmeans_model_paths :Array[String] = Array.fill[String](num_kmeans)("")
var wssse_path :String = offline_dir_path + geoTiff_dir + "/wssse"
var geotiff_hdfs_paths :Array[String] = Array.fill[String](num_kmeans)("")
var geotiff_tmp_paths :Array[String] = Array.fill[String](num_kmeans)("")

if (num_kmeans > 1) {
    var numClusters_id = 0
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        kmeans_model_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "/kmeans_model_" + numClusters + "_" + numIterations
        
        //Check if the file exists
        if (fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
            println("The kmeans model path " + kmeans_model_paths(numClusters_id) + " exists, please remove it.")
        }
        
        geotiff_hdfs_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "/clusters_" + numClusters + "_" + numIterations + ".tif"
        geotiff_tmp_paths(numClusters_id) = "/tmp/clusters_" + geoTiff_dir + "_" + numClusters + "_" + numIterations + ".tif"
        if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(numClusters_id)))) {
            println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(numClusters_id) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
        }
        numClusters_id += 1
    }
    kmeans_offline_mode = false
} else { 
    kmeans_model_paths(0) = offline_dir_path + geoTiff_dir + "/kmeans_model_" + minClusters + "_" + numIterations
    val kmeans_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(0)))
    if (kmeans_offline_mode != kmeans_offline_exists) {
        println("\"Kmeans\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + kmeans_offline_exists.toString())
        kmeans_offline_mode = kmeans_offline_exists
    }
    geotiff_hdfs_paths(0) = offline_dir_path + geoTiff_dir + "/clusters_" + minClusters + "_" + numIterations + ".tif"
    geotiff_tmp_paths(0) = "/tmp/clusters_" + geoTiff_dir + "_" + minClusters + "_" + numIterations + ".tif"
    if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(0)))) {
        println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(0) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
    }
}

single_band = false
grid0_path = hdfs:///user/emma/spring-index/BloomFinal/grid0
grid0_index_path = hdfs:///user/emma/spring-index/BloomFinal/grid0_index
grids_noNaN_path = hdfs:///user/emma/spring-index/BloomFinal/grids_noNaN
metadata_path = hdfs:///user/emma/spring-index/BloomFinal/metadata
grids_matrix_path = hdfs:///user/emma/spring-index/BloomFinal/grids_matrix
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_1091509814_36, ugi=emma (auth:SIM...


DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_1091509814_36, ugi=emma (auth:SIMPLE)]]

## Functions to (de)serialize any structure into Array[Byte]

In [59]:
def serialize(value: Any): Array[Byte] = {
    val out_stream: ByteArrayOutputStream = new ByteArrayOutputStream()
    val obj_out_stream = new ObjectOutputStream(out_stream)
    obj_out_stream.writeObject(value)
    obj_out_stream.close
    out_stream.toByteArray
}

def deserialize(bytes: Array[Byte]): Any = {
    val obj_in_stream = new ObjectInputStream(new ByteArrayInputStream(bytes))
    val value = obj_in_stream.readObject
    obj_in_stream.close
    value
}

serialize: (value: Any)Array[Byte]
deserialize: (bytes: Array[Byte])Any


## Load GeoTiffs

Using GeoTrellis all GeoTiffs of a directory will be loaded into a RDD. Using the RDD, we extract a grid from the first file to lated store the Kmeans cluster_IDS, we build an Index for populate such grid and we filter out here all NaN values.

In [60]:
//Global variables
var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var grid0: RDD[(Long, Double)] = sc.emptyRDD
var grid0_index: RDD[Long] = sc.emptyRDD
var grids_noNaN_RDD: RDD[Array[Double]] = sc.emptyRDD
var num_cols_rows :(Int, Int) = (0, 0)
var cellT :CellType = UByteCellType
var grids_RDD :RDD[Array[Double]] = sc.emptyRDD

//Local variables
val pattern: String = "tif"
val filepath: String = dir_path + geoTiff_dir

if (rdd_offline_mode) {
    grids_noNaN_RDD = sc.objectFile(grids_noNaN_path)
    grid0 = sc.objectFile(grid0_path)
    grid0_index = sc.objectFile(grid0_index_path)

    val metadata = sc.sequenceFile(metadata_path, classOf[IntWritable], classOf[BytesWritable]).map(_._2.copyBytes()).collect()
    projected_extent = deserialize(metadata(0)).asInstanceOf[ProjectedExtent]
    num_cols_rows = (deserialize(metadata(1)).asInstanceOf[Int], deserialize(metadata(2)).asInstanceOf[Int])
} else {
    if (single_band) {
        //Lets load a Singleband GeoTiffs and return RDD just with the tiles.
        val tiles_RDD = sc.hadoopGeoTiffRDD(filepath, pattern).values
    
        //Retrive the numbre of cols and rows of the Tile's grid
        val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
        val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
        num_cols_rows = (tile0.cols,tile0.rows)
        cellT = tile0.cellType
    
        grids_RDD = tiles_RDD.map(m => m.toArrayDouble())
    } else {
        //Lets load Multiband GeoTiffs and return RDD just with the tiles.
        val tiles_RDD = sc.hadoopMultibandGeoTiffRDD(filepath, pattern).values
    
        //Retrive the numbre of cols and rows of the Tile's grid
        val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
        val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
        num_cols_rows = (tile0.cols,tile0.rows)
        cellT = tile0.cellType
    
        //Lets read the average of the Spring-Index which is stored in the 4th band
        grids_RDD = tiles_RDD.map(m => m.band(3).toArrayDouble())
    }

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = sc.hadoopGeoTiffRDD(filepath, pattern).keys.zipWithIndex().map{case (e,v) => (v,e)}
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect())(0)

    //Get Index for each Cell
    val grids_withIndex = grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }
    grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => !m._1.isNaN).map { case (v, i) => (i) }

    //Get the Tile's grid
    grid0 = grids_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

    //Lets filter out NaN
    grids_noNaN_RDD = grids_RDD.map(m => m.filter(!_.isNaN))
    
    //Store data in HDFS
    grid0.saveAsObjectFile(grid0_path)
    grid0_index.saveAsObjectFile(grid0_index_path)
    grids_noNaN_RDD.saveAsObjectFile(grids_noNaN_path)
    
    val writer: SequenceFile.Writer = SequenceFile.createWriter(conf,
        Writer.file(metadata_path),
        Writer.keyClass(classOf[IntWritable]),
        Writer.valueClass(classOf[BytesWritable])
    )

    writer.append(new IntWritable(1), new BytesWritable(serialize(projected_extent)))
    writer.append(new IntWritable(2), new BytesWritable(serialize(num_cols_rows._1)))
    writer.append(new IntWritable(3), new BytesWritable(serialize(num_cols_rows._2)))
    writer.hflush()
    writer.close()
}

projected_extent = ProjectedExtent(Extent(-126.30312894720473, 14.29219617034159, -56.162671563152486, 49.25462702827337),geotrellis.proj4.CRS$$anon$3@41d0d1b7)
grid0 = MapPartitionsRDD[414] at objectFile at <console>:125
grid0_index = MapPartitionsRDD[416] at objectFile at <console>:126
grids_noNaN_RDD = MapPartitionsRDD[412] at objectFile at <console>:124
num_cols_rows = (7808,3892)
cellT = uint8raw
grids_RDD = EmptyRDD[410] at emptyRDD at <console>:115
pattern = tif
filepath = hdfs:///user/hadoop/spring-index/BloomFinal


hdfs:///user/hadoop/spring-index/BloomFinal

## Matrix

We need to do a Matrix transpose to have clusters per cell and not per year. With a GeoTiff representing a single year, the loaded data looks liks this:
```
bands_RDD.map(s => Vectors.dense(s)).cache()

//The vectors are rows and therefore the matrix will look like this:
[
Vectors.dense(0.0, 1.0, 2.0),
Vectors.dense(3.0, 4.0, 5.0),
Vectors.dense(6.0, 7.0, 8.0),
Vectors.dense(9.0, 0.0, 1.0)
]
```

To achieve that we convert the **RDD[Vector]** into a distributed Matrix, a [**CoordinateMatrix**](https://spark.apache.org/docs/latest/mllib-data-types.html#coordinatematrix), which as a **transpose** method.

In [61]:
//Global variables
var grids_matrix: RDD[Vector] = sc.emptyRDD

if (matrix_offline_mode) {
    grids_matrix = sc.objectFile(grids_matrix_path)
} else {
    val mat :RowMatrix = new RowMatrix(grids_noNaN_RDD.map(m => Vectors.dense(m)))

    // Split the matrix into one number per line.
    val byColumnAndRow = mat.rows.zipWithIndex.map {
        case (row, rowIndex) => row.toArray.zipWithIndex.map {
            case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
        }   
    }.flatMap(x => x)
    
    val matt: CoordinateMatrix = new CoordinateMatrix(byColumnAndRow)
    val matt_T = matt.transpose()
    grids_matrix = matt_T.toRowMatrix().rows
    grids_matrix.saveAsObjectFile(grids_matrix_path)
}


grids_matrix = MapPartitionsRDD[421] at objectFile at <console>:99


MapPartitionsRDD[421] at objectFile at <console>:99

## Kmeans

We use Kmeans from Sparl-MLlib. The user should only modify the variables on Kmeans setup.

### Kmeans Training

In [62]:
//Global variables
var kmeans_models :Array[KMeansModel] = new Array[KMeansModel](num_kmeans)
var wssse_data :List[(Int, Int, Double)] = List.empty

if (kmeans_offline_mode) {
    kmeans_models(0) = KMeansModel.load(sc, kmeans_model_paths(0))
    val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
    wssse_data  = wssse_data_RDD.collect().toList
} else {
    var numClusters_id = 0
    if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
        val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
        wssse_data  = wssse_data_RDD.collect().toList
        println("We will delete the file")
        try { fs.delete(new org.apache.hadoop.fs.Path(wssse_path), true) } catch { case _ : Throwable => { } }
    }
    grids_matrix.cache()
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        println(numClusters)
        kmeans_models(numClusters_id) = {
            KMeans.train(grids_matrix, numClusters, numIterations)
        }

        // Evaluate clustering by computing Within Set Sum of Squared Errors
        val WSSSE = kmeans_models(numClusters_id).computeCost(grids_matrix)
        println("Within Set Sum of Squared Errors = " + WSSSE)
                
        wssse_data = wssse_data :+ (numClusters, numIterations, WSSSE)
        
        //Save kmeans model
        kmeans_models(numClusters_id).save(sc, kmeans_model_paths(numClusters_id))
        numClusters_id += 1
    }

    //Un-persist it to save memory
    grids_matrix.unpersist()

    println("wssse_file")
    sc.parallelize(wssse_data, 1).saveAsObjectFile(wssse_path)
}

6
Within Set Sum of Squared Errors = 7.043860772070146E10                         
[Stage 213:===================================================>   (30 + 2) / 32]7
Within Set Sum of Squared Errors = 6.473463215462892E10                         
[Stage 234:=====================================================> (31 + 1) / 32]8
Within Set Sum of Squared Errors = 6.681857880633721E10                         
[Stage 255:===================================================>   (30 + 2) / 32]9
Within Set Sum of Squared Errors = 5.527455489428673E10                         
[Stage 276:===================================================>   (30 + 2) / 32]10
Within Set Sum of Squared Errors = 4.950912158685104E10                         
wssse_file                                                                      


kmeans_models = Array(org.apache.spark.mllib.clustering.KMeansModel@472d3288, org.apache.spark.mllib.clustering.KMeansModel@3b3f79b1, org.apache.spark.mllib.clustering.KMeansModel@e34b8cf, org.apache.spark.mllib.clustering.KMeansModel@570c06a6, org.apache.spark.mllib.clustering.KMeansModel@799bc7bd)
wssse_data = List((6,5,7.043860772070146E10), (7,5,6.473463215462892E10), (8,5,6.681857880633721E10), (9,5,5.527455489428673E10), (10,5,4.950912158685104E10))


List((6,5,7.043860772070146E10), (7,5,6.473463215462892E10), (8,5,6.681857880633721E10), (9,5,5.527455489428673E10), (10,5,4.950912158685104E10))

In [64]:
    if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
        var wssse_data_tmp :RDD[(Int, Int, Double)] = sc.objectFile(wssse_path)//.collect()//.toList
        println(wssse_data_tmp.collect().toList)
        println("We will delete the file")
    }

List((6,5,7.043860772070146E10), (7,5,6.473463215462892E10), (8,5,6.681857880633721E10), (9,5,5.527455489428673E10), (10,5,4.950912158685104E10))
We will delete the file


### Inspect WSSSE

In [63]:
println(wssse_data)

List((6,5,7.043860772070146E10), (7,5,6.473463215462892E10), (8,5,6.681857880633721E10), (9,5,5.527455489428673E10), (10,5,4.950912158685104E10))


### Run Kmeans clustering

Run Kmeans and obtain the clusters per each cell.

In [65]:
//Cache it so kmeans is more efficient
grids_matrix.cache()

var kmeans_res: Array[RDD[Int]] = Array.fill(num_kmeans)(sc.emptyRDD)
var numClusters_id = 0
cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    kmeans_res(numClusters_id) = kmeans_models(numClusters_id).predict(grids_matrix)
    numClusters_id += 1
}

//Un-persist it to save memory
grids_matrix.unpersist()

kmeans_res = Array(MapPartitionsRDD[607] at map at KMeansModel.scala:69, MapPartitionsRDD[608] at map at KMeansModel.scala:69, MapPartitionsRDD[609] at map at KMeansModel.scala:69, MapPartitionsRDD[610] at map at KMeansModel.scala:69, MapPartitionsRDD[611] at map at KMeansModel.scala:69)
numClusters_id = 5


MapPartitionsRDD[421] at objectFile at <console>:99

#### Sanity test

It can be skipped, it only shows the cluster ID for the first 50 cells

In [66]:
val kmeans_res_out = kmeans_res(0).take(50)
kmeans_res_out.foreach(println)

println(kmeans_res_out.size)

0
1
1
1
1
3
1
4
1
0
3
1
4
2
0
1
2
3
2
4
0
1
1
0
0
5
2
1
5
3
0
1
1
2
1
0
1
1
4
3
1
4
1
3
5
3
0
3
5
4
50


kmeans_res_out = Array(0, 1, 1, 1, 1, 3, 1, 4, 1, 0, 3, 1, 4, 2, 0, 1, 2, 3, 2, 4, 0, 1, 1, 0, 0, 5, 2, 1, 5, 3, 0, 1, 1, 2, 1, 0, 1, 1, 4, 3, 1, 4, 1, 3, 5, 3, 0, 3, 5, 4)


[0, 1, 1, 1, 1, 3, 1, 4, 1, 0, 3, 1, 4, 2, 0, 1, 2, 3, 2, 4, 0, 1, 1, 0, 0, 5, 2, 1, 5, 3, 0, 1, 1, 2, 1, 0, 1, 1, 4, 3, 1, 4, 1, 3, 5, 3, 0, 3, 5, 4]

## Build GeoTiff with Kmeans cluster_IDs

The Grid with the cluster IDs is stored in a SingleBand GeoTiff and uploaded to HDFS.

### Assign cluster ID to each grid cell and save the grid as SingleBand GeoTiff

To assign the clusterID to each grid cell it is necessary to get the indices of gird cells they belong to. The process is not straight forward because the ArrayDouble used for the creation of each dense Vector does not contain the NaN values, therefore there is not a direct between the indices in the Tile's grid and the ones in **kmeans_res** (kmeans result).

To join the two RDDS the knowledge was obtaing from a stackoverflow post on [how to perform basic joins of two rdd tables in spark using python](https://stackoverflow.com/questions/31257077/how-do-you-perform-basic-joins-of-two-rdd-tables-in-spark-using-python).

In [ ]:
var kmeans_res: Array[RDD[Int]] = Array.fill(num_kmeans)(sc.emptyRDD)
var numClusters_id = 0

cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    //Merge two RDDs, one containing the clusters_ID indices and the other one the indices of a Tile's grid cells
    val cluster_cell_pos = ((kmeans_res(numClusters_id).zipWithIndex().map{ case (v,i) => (i,v)}).join(grid0_index.zipWithIndex().map{ case (v,i) => (i,v)})).map{ case (k,(v,i)) => (v,i)}

    //Associate a Cluster_IDs to respective Grid_cell
    val grid_clusters = grid0.leftOuterJoin(cluster_cell_pos.map{ case (c,i) => (i.toLong, c)})

    //Convert all None to NaN
    val grid_clusters_res = grid_clusters.sortByKey(true).map{case (k, (v, c)) => if (c == None) (k, Double.NaN) else (k, c.get.toDouble)}
    
    //Define a Tile
    val cluster_cells :Array[Double] = grid_clusters_res.values.collect()
    val cluster_cellsD = DoubleArrayTile(cluster_cells, num_cols_rows._1, num_cols_rows._2)
    val cluster_tile = geotrellis.raster.DoubleArrayTile.empty(num_cols_rows._1, num_cols_rows._2)
    cfor(0)(_ < num_cols_rows._1, _ + 1) { col =>
        cfor(0)(_ < num_cols_rows._2, _ + 1) { row =>
            val v = cluster_cellsD.getDouble(col, row)
            cluster_tile.setDouble(col, row, v)
        }
    }

    val geoTif = new SinglebandGeoTiff(cluster_tile, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT)
    
    //Save to /tmp/
    GeoTiffWriter.write(geoTif, geotiff_tmp_paths(numClusters_id))

    //Upload to HDFS
    var cmd = "hadoop dfs -copyFromLocal -f " + geotiff_tmp_paths(numClusters_id) + " " + geotiff_hdfs_paths(numClusters_id)
    Process(cmd)!
    
    //Remove from /tmp/
    cmd = "rm -fr " + geotiff_tmp_paths(numClusters_id)
    Process(cmd)!
    
    numClusters_id += 1
}

[Stage 350:>               (0 + 8) / 45][Stage 352:>               (0 + 0) / 37]